# Detect the bottom dots on an image and use it to construct a spatial mapping

## Set number of frames, bitdepth, particle size, filename of the measurements and filename of the calibration files

In [ ]:
# number of particles (between 2 and 5)
M = 5

# number of frames
N = 1500

# bitdepth
bitdepth = 16

# particle size (Large = 1, Medium/Small = 0)
Size = 0

# startframe number
startframe = 1


m = 5 #number of row in start calibration grid
n = 6 #number of columns in start calibration grid


# filename calibration file at the start
path = 'C:/Users/s131431/Documents/Thesis/Opnames/2021-04-29/2021-04-29/2021-04-29/start_calibration.tiff'


# filename calibration file at the end
path_end ='C:/Users/s131431/Documents/Thesis/Opnames/2021-04-29/2021-04-29/2021-05-04/end_calibration.tiff'


#filename of the measurements
filesname = 'C:/Users/s131431/Documents/Thesis/Opnames/2021-05-04/2021-05-04/2021-05-04/curvelength833stroke20/measurement'
savename = '2021-05-04curvelength833stroke20.txt'

In [ ]:
# Load modules
#import cv2 
import numpy as np
import matplotlib.pyplot as plt
import cv2
import imutils
import scipy as sc
from scipy import signal

from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.dpi'] = 200

#from matplotlib.collections import LineCollection
#from matplotlib.colors import ListedColormap, BoundaryNorm




## Find the angle over which to rotate the end calibration


In [ ]:


# If image has bitdepth 8 or it has sharp contrasts making grayscale values obsolete
if bitdepth == 8:
    image = cv2.imread(path_end,cv2.CV_8UC1) * 16
# 
elif bitdepth == 16:
    image = cv2.imread(path_end,cv2.CV_16UC1)




# Fixed thresholding
meanvalue = int(np.mean(image))
thresholdvalue = meanvalue/1.5
binaryImage = cv2.threshold(image,thresholdvalue,255,cv2.THRESH_BINARY_INV)[1]


# Define a course kernel
kernelClosed = np.ones((7,7),np.uint8)
kernelOpen   = np.ones((5,5),np.uint8)

# Reduce noise inside particles
binaryImageClosed = cv2.morphologyEx(binaryImage, cv2.MORPH_CLOSE, kernelClosed)


# Reduce noise outside particles
binaryImageOpened = cv2.morphologyEx(binaryImageClosed, cv2.MORPH_OPEN, kernelOpen)
markers = cv2.connectedComponents(np.uint8(binaryImageOpened))[1]
Nmarkers = len(np.unique(markers))
Ndots = int(n*m)
maximumSize = 80
minimumSize = 30
xdots_end = np.empty(0,dtype=float)
ydots_end = np.empty(0,dtype=float)

for i in range(1,Nmarkers,1):
    idx = np.argwhere(markers==i)
    
    xSpread = np.max(idx[:,1])-np.min(idx[:,1])    
    ySpread = np.max(idx[:,0])-np.min(idx[:,0])
    
    if ((minimumSize < xSpread < maximumSize) and (minimumSize < ySpread < maximumSize)):
        xdots_end = np.append(xdots_end,np.mean(idx[:,1]))
        ydots_end = np.append(ydots_end,np.mean(idx[:,0]))
    

# Sort the dots
NRows = m
NColumns = n

for i in range(0,NRows,1):
    idx = np.argsort(xdots_end[i*NColumns:(i+1)*NColumns])
    
    xdots_end[i*NColumns:(i+1)*NColumns] = xdots_end[i*NColumns+idx]
    ydots_end[i*NColumns:(i+1)*NColumns] = ydots_end[i*NColumns+idx]
    
# Positions of the dots in meters
distanceDots = 0.02 # meters
x = np.linspace(0,(NColumns-1)*distanceDots,NColumns)
y = np.linspace(0,(NRows-1)   *distanceDots,NRows)
xReal, yReal = np.meshgrid(x,y)


Z = (ydots_end[(n-1)]-ydots_end[0])/(xdots_end[(n-1)]-xdots_end[0])

phi_end = np.arctan(Z)
phi_end = phi_end*180/np.pi

print(phi_end)

## Extract grid data from end calibration

In [ ]:

Rotation_end = phi_end

# Read image
image_end = cv2.imread(path_end)


# If image has bitdepth 8 or it has sharp contrasts making grayscale values obsolete
if bitdepth == 8:
    image_end = cv2.imread(path_end,cv2.CV_8UC1) * 16
# 
elif bitdepth == 16:
    image_end = cv2.imread(path_end,cv2.CV_16UC1)

# Show image

image_end = imutils.rotate(image_end, angle=Rotation_end)
#plt.figure()
#plt.imshow(image*1,origin='lower',cmap='gray')
#plt.show()
 

#plt.figure()
#plt.imshow(image*1,origin='lower',cmap='gray')
#plt.show()


# Fixed thresholding
meanvalue = int(np.mean(image_end))
thresholdvalue = meanvalue/1.5
binaryImage = cv2.threshold(image_end,thresholdvalue,255,cv2.THRESH_BINARY_INV)[1]


# Define a course kernel
kernelClosed = np.ones((7,7),np.uint8)
kernelOpen   = np.ones((5,5),np.uint8)

# Reduce noise inside particles
binaryImageClosed = cv2.morphologyEx(binaryImage, cv2.MORPH_CLOSE, kernelClosed)


# Reduce noise outside particles
binaryImageOpened = cv2.morphologyEx(binaryImageClosed, cv2.MORPH_OPEN, kernelOpen)
markers = cv2.connectedComponents(np.uint8(binaryImageOpened))[1]
Nmarkers = len(np.unique(markers))
Ndots = int(n*m)      # change for the number of dots on the calibration grid
maximumSize = 80
minimumSize = 30
xdots_end = np.empty(0,dtype=float)
ydots_end = np.empty(0,dtype=float)

for i in range(1,Nmarkers,1):
    idx = np.argwhere(markers==i)
    
    xSpread = np.max(idx[:,1])-np.min(idx[:,1])    
    ySpread = np.max(idx[:,0])-np.min(idx[:,0])
    
    if ((minimumSize < xSpread < maximumSize) and (minimumSize < ySpread < maximumSize)):
        xdots_end = np.append(xdots_end,np.mean(idx[:,1]))
        ydots_end = np.append(ydots_end,np.mean(idx[:,0]))
    

   
    
plt.figure()
plt.imshow(image_end,origin='lower',cmap='gray')
plt.plot(xdots_end,ydots_end,'r.',markersize=1)
plt.show()

# Sort the dots
NRows = m
NColumns = n

for i in range(0,NRows,1):
    idx = np.argsort(xdots_end[i*NColumns:(i+1)*NColumns])
    
    xdots_end[i*NColumns:(i+1)*NColumns] = xdots_end[i*NColumns+idx]
    ydots_end[i*NColumns:(i+1)*NColumns] = ydots_end[i*NColumns+idx]
    
# Positions of the dots in meters
distanceDots = 0.02 # meters
x = np.linspace(0,(NColumns-1)*distanceDots,NColumns)
y = np.linspace(0,(NRows-1)   *distanceDots,NRows)
xReal, yReal = np.meshgrid(x,y)

# We need these two arrays as input data sets for the least square fitting
Bx = xReal.flatten()
By = yReal.flatten()

# Positions of the dots in pixels
X_end = xdots_end*1 -xdots_end[0]
Y_end = ydots_end*1 -ydots_end[0]
A_end     = np.array([X_end*0+1, X_end, Y_end, X_end**2, X_end**2*Y_end, X_end**2*Y_end**2, Y_end**2, X_end*Y_end**2, X_end*Y_end]).T

# Use linear least square fitting to find the coefficients that solve A*x=B
coeffxx, _, _, _ = np.linalg.lstsq(A_end,Bx,rcond=None)
coeffyy, _, _, _ = np.linalg.lstsq(A_end,By,rcond=None)

# This is the function we just used for fitting, but here explicit
def surface(x,y, a,b,c,d,e,f,g,h,i):
    return a + b*x + c*y + d*x**2 + e*x**2*y + f*x**2*y**2 + g*y**2 + h*x*y**2 + i*x*y

# This is the function to obtain the physical coordinates from the pixel coordinates
def conversion_end(X,Y):
    X1 = surface(X,Y,*coeffxx)
    Y1 = surface(X,Y,*coeffyy)
    return (X1,Y1)
def xconversion_end(X,Y):
    X1 = surface(X,Y,*coeffxx)
    Y1 = surface(X,Y,*coeffyy)
    return(X1)
def yconversion_end(X,Y):
    X1 = surface(X,Y,*coeffxx)
    Y1 = surface(X,Y,*coeffyy)
    return(Y1)




## Plot and compare datapoints with fitted surfaces
#xplot,yplot = np.meshgrid(np.linspace(0,1500,100),np.linspace(0,1500,100))

#xplot = xplot.flatten()
#yplot = yplot.flatten()

#fig = plt.figure()
#ax = fig.add_subplot(111,projection='3d')
#ax.scatter(X_end,Y_end,xReal)
#ax.plot_trisurf(xplot,yplot,surface(xplot,yplot,*coeffxx),alpha=0.3)
#ax.set_xlabel('x img [pixels]')
#ax.set_ylabel('y img [pixels]')
#ax.set_zlabel('x real [m]')
#plt.show()

#fig = plt.figure()
#ax = fig.add_subplot(111,projection='3d')
#ax.scatter(X_end,Y_end,yReal)
#ax.plot_trisurf(xplot,yplot,surface(xplot,yplot,*coeffyy),alpha=0.3)
#ax.set_xlabel('x img [pixels]')
#ax.set_ylabel('y img [pixels]')
#ax.set_zlabel('y real [m]')
#plt.show()

## Find the angle over which to rotate the start calibration image

In [ ]:
# If image has bitdepth 8 or it has sharp contrasts making grayscale values obsolete
if bitdepth == 8:
    image = cv2.imread(path,cv2.CV_8UC1) * 16
# 
elif bitdepth == 16:
    image = cv2.imread(path,cv2.CV_16UC1)


# Fixed thresholding
meanvalue = int(np.mean(image))
thresholdvalue = meanvalue/1.5
binaryImage = cv2.threshold(image,thresholdvalue,255,cv2.THRESH_BINARY_INV)[1]


# Define a course kernel
kernelClosed = np.ones((7,7),np.uint8)
kernelOpen   = np.ones((5,5),np.uint8)

# Reduce noise inside particles
binaryImageClosed = cv2.morphologyEx(binaryImage, cv2.MORPH_CLOSE, kernelClosed)


# Reduce noise outside particles
binaryImageOpened = cv2.morphologyEx(binaryImageClosed, cv2.MORPH_OPEN, kernelOpen)
markers = cv2.connectedComponents(np.uint8(binaryImageOpened))[1]
Nmarkers = len(np.unique(markers))
Ndots = int(n*m)
maximumSize = 80
minimumSize = 30
xdots_begin = np.empty(0,dtype=float)
ydots_begin = np.empty(0,dtype=float)

for i in range(1,Nmarkers,1):
    idx = np.argwhere(markers==i)
    
    xSpread = np.max(idx[:,1])-np.min(idx[:,1])    
    ySpread = np.max(idx[:,0])-np.min(idx[:,0])
    
    if ((minimumSize < xSpread < maximumSize) and (minimumSize < ySpread < maximumSize)):
        xdots_begin = np.append(xdots_begin,np.mean(idx[:,1]))
        ydots_begin = np.append(ydots_begin,np.mean(idx[:,0]))
    

# Sort the dots
NRows = m
NColumns = n

for i in range(0,NRows,1):
    idx = np.argsort(xdots_begin[i*NColumns:(i+1)*NColumns])
    
    xdots_begin[i*NColumns:(i+1)*NColumns] = xdots_begin[i*NColumns+idx]
    ydots_begin[i*NColumns:(i+1)*NColumns] = ydots_begin[i*NColumns+idx]
    

Z2 = (ydots_begin[n-1]-ydots_begin[0])/(xdots_begin[n-1]-xdots_begin[0])

phi = np.arctan(Z2)
phi = phi*180/np.pi

print(phi)

## Extract grid data from start calibration and convert into physical coordinates

In [ ]:


# Read image
image = cv2.imread(path)


Rotation = phi

# If image has bitdepth 8 or it has sharp contrasts making grayscale values obsolete
if bitdepth == 8:
    image = cv2.imread(path,cv2.CV_8UC1) * 16
# 
elif bitdepth == 16:
    image = cv2.imread(path,cv2.CV_16UC1)

# Show image
image = imutils.rotate(image, angle=Rotation)
#plt.figure()
#plt.imshow(image*1,origin='lower',cmap='gray')
#plt.show()


#plt.figure()
#plt.imshow(image*1,origin='lower',cmap='gray')
#plt.show()

# Fixed thresholding
meanvalue = int(np.mean(image))
thresholdvalue = meanvalue/1.5
binaryImage = cv2.threshold(image,thresholdvalue,255,cv2.THRESH_BINARY_INV)[1]

# Define a course kernel
kernelClosed = np.ones((7,7),np.uint8)
kernelOpen   = np.ones((5,5),np.uint8)

# Reduce noise inside particles
binaryImageClosed = cv2.morphologyEx(binaryImage, cv2.MORPH_CLOSE, kernelClosed)

# Reduce noise outside particles
binaryImageOpened = cv2.morphologyEx(binaryImageClosed, cv2.MORPH_OPEN, kernelOpen)
markers = cv2.connectedComponents(np.uint8(binaryImageOpened))[1]
Nmarkers = len(np.unique(markers))
Ndots = int(n*m)     #change for the number of dots on the calibration grid
maximumSize = 80
minimumSize = 30
xdots_begin = np.empty(0,dtype=float)
ydots_begin = np.empty(0,dtype=float)

for i in range(1,Nmarkers,1):
    idx = np.argwhere(markers==i)
    
    xSpread = np.max(idx[:,1])-np.min(idx[:,1])    
    ySpread = np.max(idx[:,0])-np.min(idx[:,0])
    
    if ((minimumSize < xSpread < maximumSize) and (minimumSize < ySpread < maximumSize)):
        xdots_begin = np.append(xdots_begin,np.mean(idx[:,1]))
        ydots_begin = np.append(ydots_begin,np.mean(idx[:,0]))
    
plt.figure()
plt.imshow(image,origin='lower',cmap='gray')
plt.plot(xdots_begin,ydots_begin,'r.',markersize=1)
plt.show()

# Sort the dots
NRows = m
NColumns = n

for i in range(0,NRows,1):
    idx = np.argsort(xdots_begin[i*NColumns:(i+1)*NColumns])
    
    xdots_begin[i*NColumns:(i+1)*NColumns] = xdots_begin[i*NColumns+idx]
    ydots_begin[i*NColumns:(i+1)*NColumns] = ydots_begin[i*NColumns+idx]
    
# Positions of the dots in meters
distanceDots = 0.02 # meters
x = np.linspace(0,(NColumns-1)*distanceDots,NColumns)
y = np.linspace(0,(NRows-1)   *distanceDots,NRows)
xReal, yReal = np.meshgrid(x,y)

# We need these two arrays as input data sets for the least square fitting
Bx = xReal.flatten()
By = yReal.flatten()

# Positions of the dots in pixels
X = xdots_begin*1 - xdots_begin[0]
Y = ydots_begin*1 - ydots_begin[0]
A = np.array([X*0+1, X, Y, X**2, X**2*Y, X**2*Y**2, Y**2, X*Y**2, X*Y]).T

# Use linear least square fitting to find the coefficients that solve A*x=B
coeffx, _, _, _ = np.linalg.lstsq(A,Bx,rcond=None)
coeffy, _, _, _ = np.linalg.lstsq(A,By,rcond=None)

# This is the function we just used for fitting, but here explicit
def surface(x,y, a,b,c,d,e,f,g,h,i):
    return a + b*x + c*y + d*x**2 + e*x**2*y + f*x**2*y**2 + g*y**2 + h*x*y**2 + i*x*y

# This is the function to obtain the physical coordinates from the pixel coordinates
def conversion(X,Y):
    X1 = surface(X,Y,*coeffx)
    Y1 = surface(X,Y,*coeffy)
    return (X1,Y1)

def xconversion(X,Y):
    X1 = surface(X,Y,*coeffx)
    Y1 = surface(X,Y,*coeffy)
    return(X1)    
def yconversion(X,Y):
    X1 = surface(X,Y,*coeffx)
    Y1 = surface(X,Y,*coeffy)
    return(Y1)




## Plot and compare datapoints with fitted surfaces
#xplot,yplot = np.meshgrid(np.linspace(0,1500,100),np.linspace(0,1500,100))
#xplot = xplot.flatten()
#yplot = yplot.flatten()

#fig = plt.figure()
#ax = fig.add_subplot(111,projection='3d')
#ax.scatter(X,Y,xReal)
#ax.plot_trisurf(xplot,yplot,surface(xplot,yplot,*coeffx),alpha=0.3)
#ax.set_xlabel('x img [pixels]')
#ax.set_ylabel('y img [pixels]')
#ax.set_zlabel('x real [m]')
#plt.show()

#fig = plt.figure()
#ax = fig.add_subplot(111,projection='3d')
#ax.scatter(X,Y,yReal)
#ax.plot_trisurf(xplot,yplot,surface(xplot,yplot,*coeffy),alpha=0.3)
#ax.set_xlabel('x img [pixels]')
#ax.set_ylabel('y img [pixels]')
#ax.set_zlabel('y real [m]')
#plt.show()



# Particle recognition

## Find the pixel coordinates for both particles for every frame

In [ ]:
# Pixel data
if   M == 2:
    ParticlePixelx1 = np.zeros(N)
    ParticlePixely1 = np.zeros(N)
    ParticlePixelx2 = np.zeros(N)
    ParticlePixely2 = np.zeros(N)
if   M == 3:
    ParticlePixelx1 = np.zeros(N)
    ParticlePixely1 = np.zeros(N)
    ParticlePixelx2 = np.zeros(N)
    ParticlePixely2 = np.zeros(N)
    ParticlePixelx3 = np.zeros(N)
    ParticlePixely3 = np.zeros(N)
if   M == 4:
    ParticlePixelx1 = np.zeros(N)
    ParticlePixely1 = np.zeros(N)
    ParticlePixelx2 = np.zeros(N)
    ParticlePixely2 = np.zeros(N)
    ParticlePixelx3 = np.zeros(N)
    ParticlePixely3 = np.zeros(N)
    ParticlePixelx4 = np.zeros(N)
    ParticlePixely4 = np.zeros(N)
elif M == 5:
    ParticlePixelx1 = np.zeros(N)
    ParticlePixely1 = np.zeros(N)
    ParticlePixelx2 = np.zeros(N)
    ParticlePixely2 = np.zeros(N)
    ParticlePixelx3 = np.zeros(N)
    ParticlePixely3 = np.zeros(N)
    ParticlePixelx4 = np.zeros(N)
    ParticlePixely4 = np.zeros(N)
    ParticlePixelx5 = np.zeros(N)
    ParticlePixely5 = np.zeros(N)    
    
if   Size == 1:
        maximumSize = 500
elif Size == 0:
        maximumSize = 80
        
#maximumSize = 50
minimumSize = 5


# Read image
for i in range(0,N,1):
    l = i
    i = i+ startframe
    x = str(i)
    x = x.zfill(4)
    filename = filesname + x + '.tiff'
    #filename = filesname + x + '.tif'

    image = cv2.imread(filename)
    #image = cv2.imread(filename,cv2.CV_16UC1)
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    
    
    # noise removal
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

    #plt.figure()
    #plt.imshow(opening,origin='lower',cmap='gray')
    #plt.show()

    # sure background area
    sure_bg = cv2.dilate(opening,kernel,iterations=3)



    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
    ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
    
   
    
    dist_transform_1d = np.max(dist_transform,axis=1)
    y_coordinates = (sc.signal.find_peaks(dist_transform_1d))
    y_array = np.asarray(y_coordinates[0])
    
    if M == 2:
        ParticlePixely1[l] = y_array[0]
        ParticlePixely2[l] = y_array[1]
        A = ParticlePixely1[l]
        A = A.astype(int)
        B = ParticlePixely2[l]
        B = B.astype(int)
    
    if M == 3:
        ParticlePixely1[l] = y_array[0]
        ParticlePixely2[l] = y_array[1]
        ParticlePixely3[l] = y_array[2]  
        A = ParticlePixely1[l]
        A = A.astype(int)
        B = ParticlePixely2[l]
        B = B.astype(int)
        C = ParticlePixely3[l]
        C = C.astype(int)
    
    if M == 4:
        ParticlePixely1[l] = y_array[0]
        ParticlePixely2[l] = y_array[1]
        ParticlePixely3[l] = y_array[2]
        ParticlePixely4[l] = y_array[3] 
        A = ParticlePixely1[l]
        A = A.astype(int)
        B = ParticlePixely2[l]
        B = B.astype(int)
        C = ParticlePixely3[l]
        C = C.astype(int)
        D = ParticlePixely4[l]
        D = D.astype(int)
    
    elif M == 5:
        ParticlePixely1[l] = y_array[0]
        ParticlePixely2[l] = y_array[1]
        ParticlePixely3[l] = y_array[2]
        ParticlePixely4[l] = y_array[3]
        ParticlePixely5[l] = y_array[4] 
        A = ParticlePixely1[l]
        A = A.astype(int)
        B = ParticlePixely2[l]
        B = B.astype(int)
        C = ParticlePixely3[l]
        C = C.astype(int)
        D = ParticlePixely4[l]
        D = D.astype(int)
        E = ParticlePixely5[l]
        E = C.astype(int)
    
    #dist_transform = dist_transform[:][:].astype(int)
    
 
    
    
    #K = np.zeros(N)
    
    K = np.max(dist_transform[:][A])
    result = np.where(dist_transform[:][A] == K)
    result = np.asarray(result[0])
    ParticlePixelx1[l] = result[0]
    
    L = np.max(dist_transform[:][B])
    result = np.where(dist_transform[:][B] == L)
    result = np.asarray(result[0])
    ParticlePixelx2[l] = result[0]
    
    if M > 2:
        O = np.max(dist_transform[:][C])
        result = np.where(dist_transform[:][C] == O)
        result = np.asarray(result[0])
        ParticlePixelx3[l] = result[0]
    
    if M > 3:
        P = np.max(dist_transform[:][D])
        result = np.where(dist_transform[:][D] == P)
        result = np.asarray(result[0])
        ParticlePixelx4[l] = result[0]
    
    if M > 4:
        Q = np.max(dist_transform[:][E])
        result = np.where(dist_transform[:][E] == Q)
        result = np.asarray(result[0])
        ParticlePixelx5[l] = result[0]
    

    #plt.figure()
    #plt.imshow(dist_transform,origin='lower',cmap='gray')
    #plt.show()
    
    # Find marker dots
    #markers = 2
    #markers = cv2.connectedComponents(np.uint8(opening))[1]
    #Nmarkers = len(np.unique(markers))
    #Ndots = int(2)
    
    #xdots = np.empty(0,dtype=float)
    #ydots = np.empty(0,dtype=float)
    
    #for k in range(1,Nmarkers,1):
     #   idxx = np.argwhere(markers==k)
    
      #  xSpread = np.max(idxx[:,1])-np.min(idxx[:,1])    
       # ySpread = np.max(idxx[:,0])-np.min(idxx[:,0])
    
        #if ((minimumSize < xSpread < maximumSize) and (minimumSize < ySpread < maximumSize)):
         #   xdots = np.append(xdots,np.mean(idxx[:,1]))
          #  ydots = np.append(ydots,np.mean(idxx[:,0]))
    
    #ParticlePixelx1[l] = xdots[0]
    #ParticlePixely1[l] = ydots[0]
    #ParticlePixelx2[l] = xdots[1]
    #ParticlePixely2[l] = ydots[1] 
    
    
   
       
# check the last frame    
plt.figure()
plt.imshow(image,origin='lower',cmap='gray')
plt.plot(ParticlePixelx1[N-1],ParticlePixely1[N-1],'r.',markersize=3)
plt.plot(ParticlePixelx2[N-1],ParticlePixely2[N-1],'r.',markersize=3)
if M > 2:
    plt.plot(ParticlePixelx3[N-1],ParticlePixely3[N-1],'r.',markersize=3)
if M > 3:
    plt.plot(ParticlePixelx4[N-1],ParticlePixely4[N-1],'r.',markersize=3)
if M > 4:
    plt.plot(ParticlePixelx5[N-1],ParticlePixely5[N-1],'r.',markersize=3)
plt.grid()
plt.show()



## Convert pixel coordinates to physical coordinates using the start calibration and plot the data

In [ ]:
# Physical coordinates from the start calibration
Particlex1 = np.zeros(N)
Particley1 = np.zeros(N)
Particlex2 = np.zeros(N)
Particley2 = np.zeros(N)
if M > 2:
    Particlex3 = np.zeros(N)
    Particley3 = np.zeros(N)
if M > 3:
    Particlex4 = np.zeros(N)
    Particley4 = np.zeros(N)
if M > 4:
    Particlex5 = np.zeros(N)
    Particley5 = np.zeros(N)


#x_correction_end = xconversion_end(X_end[0],Y_end[0])
#y_correction_end = yconversion_end(X_end[0],Y_end[0])
#x_correction = xconversion(X[0],Y[0])
#y_correction = yconversion(X[0],Y[0])


Particlex1 = xconversion(ParticlePixelx1, ParticlePixely1) #- x_correction
Particley1 = yconversion(ParticlePixelx1, ParticlePixely1) #- y_correction
Particlex2 = xconversion(ParticlePixelx2, ParticlePixely2) #- x_correction
Particley2 = yconversion(ParticlePixelx2, ParticlePixely2) #- y_correction
if M > 2:
    Particlex3 = xconversion(ParticlePixelx3, ParticlePixely3) #- x_correction
    Particley3 = yconversion(ParticlePixelx3, ParticlePixely3) #- y_correction
if M > 3:
    Particlex4 = xconversion(ParticlePixelx4, ParticlePixely4) #- x_correction
    Particley4 = yconversion(ParticlePixelx4, ParticlePixely4) #- y_correction
if M > 4:
    Particlex5 = xconversion(ParticlePixelx5, ParticlePixely5) #- x_correction
    Particley5 = yconversion(ParticlePixelx5, ParticlePixely5) #- y_correction



#plt.figure()
#plt.plot(Particlex1,Particley1)
#plt.plot(Particlex1[0], Particley1[0], 'r.')
#plt.xlabel('x coordinate [m]')
#plt.ylabel('y coordinate [m]')
#plt.legend(['Particle1', 'Start point'])
#plt.grid()
#plt.show()


#plt.figure()
#plt.plot(Particlex2,Particley2)
#plt.plot(Particlex2[0], Particley2[0], 'r.')
#plt.xlabel('x coordinate [m]')
#plt.ylabel('y coordinate [m]')
#plt.legend(['Particle2', 'Start point'])
#plt.grid()
#plt.show()


plt.figure()
plt.plot(Particlex1,Particley1)
plt.plot(Particlex1[0], Particley1[0], 'r.')
plt.plot(Particlex2,Particley2)
plt.plot(Particlex2[0], Particley2[0], 'r.')
if M > 2:
    plt.plot(Particlex3,Particley3)
    plt.plot(Particlex3[0], Particley3[0], 'r.')
if M > 3:
    plt.plot(Particlex4,Particley4)
    plt.plot(Particlex4[0], Particley4[0], 'r.')
if M > 4:
    plt.plot(Particlex5,Particley5)
    plt.plot(Particlex5[0], Particley5[0], 'r.')
plt.xlabel('x coordinate [m]')
plt.ylabel('y coordinate [m]')
#plt.legend(['Particle1', 'Start point 1', 'Particle2','Start point 2'])
plt.grid()
plt.show()

## Check particles at the last frame

## Find difference between particle coordinates using the start and the end calibration

In [ ]:
# physical coordinates using the end calibration
Particlex1_end = xconversion_end(ParticlePixelx1, ParticlePixely1) #- x_correction_end
Particley1_end = yconversion_end(ParticlePixelx1, ParticlePixely1) #- y_correction_end
Particlex2_end = xconversion_end(ParticlePixelx2, ParticlePixely2) #- x_correction_end
Particley2_end = yconversion_end(ParticlePixelx2, ParticlePixely2) #- y_correction_end
if M > 2:
    Particlex3_end = xconversion_end(ParticlePixelx3, ParticlePixely3) #- x_correction_end
    Particley3_end = yconversion_end(ParticlePixelx3, ParticlePixely3) #- y_correction_end
if M > 3:
    Particlex4_end = xconversion_end(ParticlePixelx4, ParticlePixely4) #- x_correction_end
    Particley4_end = yconversion_end(ParticlePixelx4, ParticlePixely4) #- y_correction_end
if M > 4:
    Particlex5_end = xconversion_end(ParticlePixelx5, ParticlePixely5) #- x_correction_end
    Particley5_end = yconversion_end(ParticlePixelx5, ParticlePixely5) #- y_correction_end

#Particle1_end = conversion_end(ParticlePixelx1, ParticlePixely1)
#Particle2_end = conversion_end(ParticlePixelx2, ParticlePixely2)


#physical coordinates suing the start calibration
#Particle1 = np.zeros((2,N))
#Particle2 = np.zeros((2,N))

#Particle1 = conversion(ParticlePixelx1, ParticlePixely1)
#Particle2 = conversion(ParticlePixelx2, ParticlePixely2) 


# difference from start and end calibration
xdifference1 = Particlex1 - Particlex1_end 
ydifference1 = Particley1 - Particley1_end
xdifference2 = Particlex2 - Particlex2_end
ydifference2 = Particley2 - Particley2_end
if M > 2:
    xdifference3 = Particlex3 - Particlex3_end
    ydifference3 = Particley3 - Particley3_end
if M > 3:
    xdifference4 = Particlex4 - Particlex4_end
    ydifference4 = Particley4 - Particley4_end
if M > 4:
    xdifference5 = Particlex5 - Particlex5_end
    ydifference5 = Particley5 - Particley5_end

#ydifference1 = Particley1 - Particley1_end
#xdifference2 = Particle2[0] - Particle2_end[0]
#ydifference2 = Particle2[1] - Particle2_end[1]


#print(Particle1)
#print(Particle2)
#print(Particle1_end)
#print(Particle2_end)

print(np.mean(xdifference1))
print(np.mean(ydifference1))
print(np.mean(xdifference2))
print(np.mean(ydifference2))
if M > 2:
    print(np.mean(xdifference3))
    print(np.mean(ydifference3))
if M > 3:
    print(np.mean(xdifference4))
    print(np.mean(ydifference4))
if M > 4:
    print(np.mean(xdifference5))
    print(np.mean(ydifference5))


## Settings for the graphs

In [ ]:
#Want to include gap between first and last particle? (No -> 0, Yes -> 1)
Include_totalgap = 1

#Want to include gap between neighbouring particles? (No -> 0, Yes -> 1)
Include_neighbourgap = 1

#Dotsize
Dotsize = 10

## Fit plots in order to correct for translational and rotational effects

In [ ]:
x12 = Particlex2 - Particlex1
y12 = Particley2 - Particley1
if M > 2:
    x23 = Particlex3 - Particlex2
    y23 = Particley3 - Particley2
    x13 = Particlex3 - Particlex1
    y13 = Particley3 - Particley1
if M > 3:
    x34 = Particlex4 - Particlex3
    y34 = Particley4 - Particley3
    x14 = Particlex4 - Particlex1
    y14 = Particley4 - Particley1
if M > 4:
    x45 = Particlex5 - Particlex4
    y45 = Particley5 - Particley4
    x15 = Particlex5 - Particlex1
    y15 = Particley5 - Particley1


if M == 2:
    cm12 = plt.cm.get_cmap('Greys')
if M == 3:
    cm12 = plt.cm.get_cmap('Reds')
    cm23 = plt.cm.get_cmap('Blues')
    cm13 = plt.cm.get_cmap('Greys')
if M == 4:
    cm12 = plt.cm.get_cmap('Reds')
    cm23 = plt.cm.get_cmap('Blues')
    cm34 = plt.cm.get_cmap('Greens')
    cm14 = plt.cm.get_cmap('Greys')
if M == 5:
    cm12 = plt.cm.get_cmap('Reds')
    cm23 = plt.cm.get_cmap('Blues')
    cm34 = plt.cm.get_cmap('Greens')
    cm45 = plt.cm.get_cmap('Oranges')
    cm15 = plt.cm.get_cmap('Greys')

    
    
framenumber = range(N)
z = framenumber
if M == 2:
    if Include_neighbourgap == 1:
        sc12 = plt.scatter(x12, y12, c=z, vmin=0, vmax=N, s=Dotsize, cmap=cm12)
        plt.colorbar(sc12)
if M == 3:
    if Include_neighbourgap == 1:
        sc12 = plt.scatter(x12, y12, c=z, vmin=0, vmax=N, s=Dotsize, cmap=cm12)
        plt.colorbar(sc12)
        sc23 = plt.scatter(x23, y23, c=z, vmin=0, vmax=N, s=Dotsize, cmap=cm23)
        plt.colorbar(sc23)
    if Include_totalgap == 1:
        sc13 = plt.scatter(x13, y13, c=z, vmin=0, vmax=N, s=Dotsize, cmap=cm13)
        plt.colorbar(sc13)
if M == 4:
    if Include_neighbourgap == 1:
        sc12 = plt.scatter(x12, y12, c=z, vmin=0, vmax=N, s=Dotsize, cmap=cm12)
        plt.colorbar(sc12)
        sc23 = plt.scatter(x23, y23, c=z, vmin=0, vmax=N, s=Dotsize, cmap=cm23)
        plt.colorbar(sc23)
        sc34 = plt.scatter(x34, y34, c=z, vmin=0, vmax=N, s=Dotsize, cmap=cm34)
        plt.colorbar(sc34)
    if Include_totalgap == 1:
        sc14 = plt.scatter(x14, y14, c=z, vmin=0, vmax=N, s=Dotsize, cmap=cm14)
        plt.colorbar(sc14)      
        
if M == 5:
    if Include_neighbourgap == 1:
        sc12 = plt.scatter(x12, y12, c=z, vmin=0, vmax=N, s=Dotsize, cmap=cm12)
        plt.colorbar(sc12)
        sc23 = plt.scatter(x23, y23, c=z, vmin=0, vmax=N, s=Dotsize, cmap=cm23)
        plt.colorbar(sc23)
        sc34 = plt.scatter(x34, y34, c=z, vmin=0, vmax=N, s=Dotsize, cmap=cm34)
        plt.colorbar(sc34)            
        sc45 = plt.scatter(x45, y45, c=z, vmin=0, vmax=N, s=Dotsize, cmap=cm45)
        plt.colorbar(sc45)
    if Include_totalgap == 1:
        sc15 = plt.scatter(x15, y15, c=z, vmin=0, vmax=N, s=Dotsize, cmap=cm15)
        plt.colorbar(sc15)

#plt.plot(x,y, 'y', linewidth=0.5)
plt.xlabel('x distance between particle centers')
plt.ylabel('y distance between particle centers')
plt.grid()
plt.show()


#plt.figure()

#plt.plot(x,y,'o')
#plt.show()

#fig, ax = plt.subplots()
#im = ax.scatter(x, y, c=c)
#fig.colorbar(im, ax=ax)

#print(np.mean(x))
#print(np.mean(y))
#print(np.max(y))
#print(np.min(y))

In [ ]:
# fit plots
plt.figure()

plt.plot(Particlex1, Particley1)
#plt.plot(Particlex2-Particlex2[0],Particley2-Particley2[0])

plt.plot(Particlex1_end + (Particlex1[0]-Particlex1_end[0]), Particley1_end + (Particley1[0] - Particley1_end[0]))
#plt.plot(Particlex2_end-Particlex2_end[0],Particley2_end-Particley2_end[0])




plt.xlabel('x coordinate [m]')
plt.ylabel('y coordinate [m]')
#plt.legend(['Particle1', 'Particle1_end', 'Particle1_end','Particle2_end'])
plt.grid()
plt.show()

## Data of the gap between the two particles - Absolute distance

In [ ]:
x12 = Particlex2 - Particlex1
y12 = Particley2 - Particley1
if M > 2:
    x23 = Particlex3 - Particlex2
    y23 = Particley3 - Particley2
    x13 = Particlex3 - Particlex1
    y13 = Particley3 - Particley1
if M > 3:
    x34 = Particlex4 - Particlex3
    y34 = Particley4 - Particley3
    x14 = Particlex4 - Particlex1
    y14 = Particley4 - Particley1
if M > 4:
    x45 = Particlex5 - Particlex4
    y45 = Particley5 - Particley4
    x15 = Particlex5 - Particlex1
    y15 = Particley5 - Particley1
    
distancelength12 = np.sqrt(x12**2+y12**2)
if M > 2:
    distancelength23 = np.sqrt(x23**2+y23**2)
    distancelength13 = np.sqrt(x13**2+y13**2)
if M > 3:
    distancelength34 = np.sqrt(x34**2+y34**2)
    distancelength14 = np.sqrt(x14**2+y14**2)
if M > 4:
    distancelength45 = np.sqrt(x45**2+y45**2)
    distancelength15 = np.sqrt(x15**2+y15**2)

framenumber = range(N)
if M == 2:
    if Include_neighbourgap == 1:
        sc = plt.scatter(framenumber, distancelength12,  s=Dotsize)
if M == 3:
    if Include_neighbourgap == 1:
        sc = plt.scatter(framenumber, distancelength12,  s=Dotsize)
        sc = plt.scatter(framenumber, distancelength23,  s=Dotsize)
    if Include_totalgap == 1:
        sc = plt.scatter(framenumber, distancelength13,  s=Dotsize)
if M == 4:
    if Include_neighbourgap == 1:
        sc = plt.scatter(framenumber, distancelength12,  s=Dotsize)
        sc = plt.scatter(framenumber, distancelength23,  s=Dotsize)
        sc = plt.scatter(framenumber, distancelength34,  s=Dotsize)
    if Include_totalgap == 1:
        sc = plt.scatter(framenumber, distancelength14,  s=Dotsize)
if M == 5:
    if Include_neighbourgap == 1:
        sc = plt.scatter(framenumber, distancelength12,  s=Dotsize)
        sc = plt.scatter(framenumber, distancelength23,  s=Dotsize)
        sc = plt.scatter(framenumber, distancelength34,  s=Dotsize)
        sc = plt.scatter(framenumber, distancelength45,  s=Dotsize)
    if Include_totalgap == 1:
        sc = plt.scatter(framenumber, distancelength15,  s=Dotsize)
#sc = plt.scatter(framenumber, y,  s=10)

plt.xlabel('frame number')
plt.ylabel('absolute distance between particle centers')
plt.grid()
plt.show()




#plt.figure()

#plt.plot(x,y,'o')
#plt.show()

#fig, ax = plt.subplots()
#im = ax.scatter(x, y, c=c)
#fig.colorbar(im, ax=ax)

## Data of the gap between the particles - dy

In [ ]:
x12 = Particlex2 - Particlex1
y12 = Particley2 - Particley1
if M > 2:
    x23 = Particlex3 - Particlex2
    y23 = Particley3 - Particley2
    x13 = Particlex3 - Particlex1
    y13 = Particley3 - Particley1
if M > 3:
    x34 = Particlex4 - Particlex3
    y34 = Particley4 - Particley3
    x14 = Particlex4 - Particlex1
    y14 = Particley4 - Particley1
if M > 4:
    x45 = Particlex5 - Particlex4
    y45 = Particley5 - Particley4
    x15 = Particlex5 - Particlex1
    y15 = Particley5 - Particley1

    
if M == 2:
    std_y12 = np.std(y12)
    std_distancelength12 = np.std(distancelength12)
    y_meangap12 =  format(np.mean(y12),'.6f')
    std_y_meangap12 = format(np.mean(std_y12),'.6f')
    abs_meangap12 = format(np.mean(distancelength12),'.6f')
    std_abs_meangap12 = format(np.mean(std_distancelength),'.6f')
    
if M == 3:
    std_y12 = np.std(y12)
    std_y23 = np.std(y23)
    std_y13 = np.std(y13)
    std_distancelength12 = np.std(distancelength12)
    std_distancelength23 = np.std(distancelength23)
    std_distancelength13 = np.std(distancelength13)
    y_meangap12 =  format(np.mean(y12),'.6f')
    y_meangap23 =  format(np.mean(y23),'.6f')
    y_meangap13 =  format(np.mean(y13),'.6f')
    std_y_meangap12 = format(np.mean(std_y12),'.6f')
    std_y_meangap23 = format(np.mean(std_y23),'.6f')
    std_y_meangap13 = format(np.mean(std_y13),'.6f')
    abs_meangap12 = format(np.mean(distancelength12),'.6f')
    abs_meangap23 = format(np.mean(distancelength23),'.6f')
    abs_meangap13 = format(np.mean(distancelength13),'.6f')
    std_abs_meangap12 = format(np.mean(std_distancelength12),'.6f')
    std_abs_meangap23 = format(np.mean(std_distancelength23),'.6f')
    std_abs_meangap13 = format(np.mean(std_distancelength13),'.6f')
    
if M == 4:
    std_y12 = np.std(y12)
    std_y23 = np.std(y23)
    std_y34 = np.std(y34)
    std_y14 = np.std(y14)
    std_distancelength12 = np.std(distancelength12)
    std_distancelength23 = np.std(distancelength23)
    std_distancelength34 = np.std(distancelength34)
    std_distancelength14 = np.std(distancelength14)
    y_meangap12 =  format(np.mean(y12),'.6f')
    y_meangap23 =  format(np.mean(y23),'.6f')
    y_meangap34 =  format(np.mean(y34),'.6f')
    y_meangap14 =  format(np.mean(y14),'.6f')
    std_y_meangap12 = format(np.mean(std_y12),'.6f')
    std_y_meangap23 = format(np.mean(std_y23),'.6f')
    std_y_meangap34 = format(np.mean(std_y34),'.6f')
    std_y_meangap14 = format(np.mean(std_y14),'.6f')
    abs_meangap12 = format(np.mean(distancelength12),'.6f')
    abs_meangap23 = format(np.mean(distancelength23),'.6f')
    abs_meangap34 = format(np.mean(distancelength34),'.6f')
    abs_meangap14 = format(np.mean(distancelength14),'.6f')
    std_abs_meangap12 = format(np.mean(std_distancelength12),'.6f')
    std_abs_meangap23 = format(np.mean(std_distancelength23),'.6f')
    std_abs_meangap34 = format(np.mean(std_distancelength34),'.6f')
    std_abs_meangap14 = format(np.mean(std_distancelength14),'.6f')
    
if M == 5:
    std_y12 = np.std(y12)
    std_y23 = np.std(y23)
    std_y34 = np.std(y34)
    std_y45 = np.std(y45)
    std_y15 = np.std(y15)
    std_distancelength12 = np.std(distancelength12)
    std_distancelength23 = np.std(distancelength23)
    std_distancelength34 = np.std(distancelength34)
    std_distancelength45 = np.std(distancelength45)
    std_distancelength15 = np.std(distancelength15)
    y_meangap12 =  format(np.mean(y12),'.6f')
    y_meangap23 =  format(np.mean(y23),'.6f')
    y_meangap34 =  format(np.mean(y34),'.6f')
    y_meangap45 =  format(np.mean(y45),'.6f')
    y_meangap15 =  format(np.mean(y15),'.6f')
    std_y_meangap12 = format(np.mean(std_y12),'.6f')
    std_y_meangap23 = format(np.mean(std_y23),'.6f')
    std_y_meangap34 = format(np.mean(std_y34),'.6f')
    std_y_meangap45 = format(np.mean(std_y45),'.6f')
    std_y_meangap15 = format(np.mean(std_y15),'.6f')
    abs_meangap12 = format(np.mean(distancelength12),'.6f')
    abs_meangap23 = format(np.mean(distancelength23),'.6f')
    abs_meangap34 = format(np.mean(distancelength34),'.6f')
    abs_meangap45 = format(np.mean(distancelength45),'.6f')
    abs_meangap15 = format(np.mean(distancelength15),'.6f')
    std_abs_meangap12 = format(np.mean(std_distancelength12),'.6f')
    std_abs_meangap23 = format(np.mean(std_distancelength23),'.6f')
    std_abs_meangap34 = format(np.mean(std_distancelength34),'.6f')
    std_abs_meangap45 = format(np.mean(std_distancelength45),'.6f')
    std_abs_meangap15 = format(np.mean(std_distancelength15),'.6f')
    
#print('y distance = ',format(np.mean(y),'.6f'))
#print('std y_distance = ',format(np.mean(std_y),'.6f'))
#print('absolute distance = ', format(np.mean(distancelength),'.6f'))
#print('std absolute distance = ', format(np.mean(std_distancelength),'.6f'))


framenumber = range(N)
if M == 2:
    sc = plt.scatter(framenumber, distancelength12,  s=Dotsize)
    sc = plt.scatter(framenumber, y,  s=Dotsize)
if M == 3:
    sc = plt.scatter(framenumber, distancelength12+distancelength23, s=Dotsize)
    sc = plt.scatter(framenumber, distancelength13, s=Dotsize)
if M == 4:
    sc = plt.scatter(framenumber, distancelength12+distancelength23+distancelength34, s=Dotsize)
    sc = plt.scatter(framenumber, distancelength14, s=Dotsize)
if M == 5:
    sc = plt.scatter(framenumber, distancelength12+distancelength23+distancelength34+distancelength45, s=Dotsize)
    sc = plt.scatter(framenumber, distancelength15, s=Dotsize)
plt.xlabel('frame number')
plt.ylabel('total gap')
plt.grid()
plt.show()

In [ ]:
# set linewidth
Set_linewidth = 4

framenumber = range(N)
if M == 2:
    sc = plt.plot(framenumber, x12,  linewidth = Set_linewidth)
if M == 3:
    sc = plt.plot(framenumber, x12, linewidth = Set_linewidth)
    sc = plt.plot(framenumber, x23, linewidth = Set_linewidth)
    sc = plt.plot(framenumber, x13, linewidth = Set_linewidth)
if M == 4:
    sc = plt.plot(framenumber, x12, linewidth = Set_linewidth)
    sc = plt.plot(framenumber, x23, linewidth = Set_linewidth)
    sc = plt.plot(framenumber, x34, linewidth = Set_linewidth)
    sc = plt.plot(framenumber, x14, linewidth = Set_linewidth)
if M == 5:
    sc = plt.plot(framenumber, x12, linewidth = Set_linewidth)
    sc = plt.plot(framenumber, x23, linewidth = Set_linewidth)
    sc = plt.plot(framenumber, x34, linewidth = Set_linewidth)
    sc = plt.plot(framenumber, x45, linewidth = Set_linewidth)
    sc = plt.plot(framenumber, x15, linewidth = Set_linewidth)
plt.xlabel('frame number')
plt.ylabel('x-gap')
plt.legend(['x12', 'x23', 'x34','x45','x15'])
plt.grid()
plt.show()

## Save the data

In [ ]:
std_yy = np.full((N, 1), std_y)
std_distancelengthh = np.full((N,1), std_distancelength)

DataOut = np.column_stack((framenumber,y,std_yy,distancelength,std_distancelengthh))
#np.savetxt('output.txt', DataOut)
np.savetxt(savename, DataOut, fmt=('%i','%20.8f','%20.8f','%20.8f','%20.8f'))

## Load the data

In [ ]:
framenumber,y,std_y,distancelength,std_distancelength = np.loadtxt(savename, unpack=True)
#print(framenumber)
#print(distancelength)
#print(std_y)